<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Learning FourSquare API with Python</font></h1>


## Introduction

In this lab, you will learn in details how to make calls to the Foursquare API for different purposes. You will learn how to construct a URL to send a request to the API to search for a specific type of venues, to explore a particular venue, to explore a Foursquare user, to explore a geographical location, and to get trending venues around a location. Also, you will learn how to use the visualization library, Folium, to visualize the results.


## Table of Contents

1.  <a href="#item1">Foursquare API Search Function</a>
2.  <a href="#item2">Explore a Given Venue</a>  
3.  <a href="#item3">Explore a User</a>  
4.  <a href="#item4">Foursquare API Explore Function</a>  
5.  <a href="#item5">Get Trending Venues</a>  


### Import necessary Libraries


In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy   # geopy is a Python client for several popular geocoding web services
from geopy.geocoders import Nominatim #module to convert an address into latitude and longitude values (OpenStreetMap geocoding service)

# (from geopy.geocoders import GoogleV3) would be the codeline if we wanted to use Google Maps Platform as geocoding service

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file (dict or list) into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print()
print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 112kB 19.4MB/s eta 0:00:01

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version


##### Make sure that you have created a Foursquare developer account and have your credentials handy


In [2]:
CLIENT_ID = '113ZWQSUYLUGMKP51IRZVZRVDHJBACXCYTXIXGD5GM5PVO5B' # your Foursquare ID
CLIENT_SECRET = '5QXPGLDYT1SVGN3FVVLMZEV0EKJMYPMJNMJAMF2X3ZZEW0SP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 113ZWQSUYLUGMKP51IRZVZRVDHJBACXCYTXIXGD5GM5PVO5B
CLIENT_SECRET:5QXPGLDYT1SVGN3FVVLMZEV0EKJMYPMJNMJAMF2X3ZZEW0SP


#### Let's again assume that you are staying at the Conrad Hotel.
##### So let's get started by converting the Conrad Hotel's address to its latitude and longitude coordinates.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ** <em>foursquare_agent</em> **, as shown below.It could be any name, but it is not advisable to go on with a generic 'user_agent'. In other words, we need to create an object-instance to make good use of all methods of a Class. The syntax below is pretty straightforward and defined by "https://geopy.readthedocs.io/en/stable/#module-geopy.geocoders".


In [3]:
address = '102 North End Ave, New York, NY' # the hotel address could have been inserted directly as a parameter
print(type(address))
print()

geolocator = Nominatim(user_agent="foursquare_agent") # documentation strongly suggests to define a user_agent (OSM only)
print(type(geolocator))
print()

location = geolocator.geocode(address) # this is the most important line as it defines the location we will be making searches from

latitude = location.latitude
longitude = location.longitude
cru = location.raw                  # this variable brings all OSM parameters about this venue as a dictionary 
print(latitude, longitude)
print()
print(cru)
print()
print(type(cru))

<class 'str'>

<class 'geopy.geocoders.nominatim.Nominatim'>

40.7149555 -74.0153365

{'place_id': 59325501, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 4957663866, 'boundingbox': ['40.7149055', '40.7150055', '-74.0153865', '-74.0152865'], 'lat': '40.7149555', 'lon': '-74.0153365', 'display_name': 'Loopy Doopy Rooftop Bar, 102, North End Avenue, Battery Park City, Manhattan Community Board 1, Manhattan, New York County, New York, 10282, United States of America', 'class': 'amenity', 'type': 'bar', 'importance': 0.611, 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//food_bar.p.20.png'}

<class 'dict'>


<a id="item1"></a>


## 1. Search for a specific venue category

> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

#### Now, let's assume that it is lunch time, and you are craving Italian food. So, let's define a query to search for Italian food that is within 500 metres from the Conrad Hotel.


In [4]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


#### Define the corresponding URL


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
print(url)
print(type(url))

https://api.foursquare.com/v2/venues/search?client_id=113ZWQSUYLUGMKP51IRZVZRVDHJBACXCYTXIXGD5GM5PVO5B&client_secret=5QXPGLDYT1SVGN3FVVLMZEV0EKJMYPMJNMJAMF2X3ZZEW0SP&ll=40.7149555,-74.0153365&v=20180604&query=Italian&radius=500&limit=30
<class 'str'>


#### Send the GET Request and examine the results
</p> The get() method sends a GET request to the specified url. It makes a request to a web page, and return the status code which can be a json file for example.

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fbfaa015f561c659f5fd003'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351},
      {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
     'distance': 58,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-16063964

In [7]:
type(results)

dict

In [8]:
list(results.keys())

['meta', 'response']

In [19]:
# list(results.values())

Right now, taking into account that this Lab Session is a learning session, it is fair to explore the very nature of what **"results"**  is:
</p> a) **Results** is a Dictionary that displays two keys only (meta and response);
</p> b) However, some of these 2 keys' values are also dictionaries on their own, for example, meta itself;
</p> c)  Within these "inner" dictionaries, there are other dictionaires as well, for example, response is followed by"{'venues': [{'id': '4fa862b3e4b0ebff2f749f06','name': "Harry's Italian Pizza Bar",    'location': {'address': '225 Murray St'.......etc

#### Get relevant part of JSON and transform it into a _pandas_ dataframe
</p> The first key (meta) displays data not required to the next steps (200 means process worked (opposite is 404) and the other is just request identification).
</p> The information we need to extract from the huge output-dictionary as result of the query (search) are contained in the key 'response' and "sub-key" venue.

In [9]:
# assign relevant part of JSON to venues
venues = results['response']['venues']
print(type(venues))
print()
print(venues)
print()

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

<class 'list'>

[{'id': '4fa862b3e4b0ebff2f749f06', 'name': "Harry's Italian Pizza Bar", 'location': {'address': '225 Murray St', 'lat': 40.71521779064671, 'lng': -74.01473940209351, 'labeledLatLngs': [{'label': 'display', 'lat': 40.71521779064671, 'lng': -74.01473940209351}, {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}], 'distance': 58, 'postalCode': '10282', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['225 Murray St', 'New York, NY 10282', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d1ca941735', 'name': 'Pizza Place', 'pluralName': 'Pizza Places', 'shortName': 'Pizza', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_', 'suffix': '.png'}, 'primary': True}], 'referralId': 'v-1606396417', 'hasPerk': False}, {'id': '4f3232e219836c91c7bfde94', 'name': 'Conca Cucina Italian Restaurant', 'location': {'address': '63 W Broadway', 'lat': 40.714484000000006, 'lng': -74.00980600000001, 'labeledLat

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1606396417,False,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1606396417,False,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN
2,3fd66200f964a520f4e41ee3,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1606396417,False,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway


In [10]:
dataframe.shape

(3, 17)

**Very Important Note:**
A test aimed at better understanding **results** structure can be performed by defining limit=100 in order to select only the 1st restaurant. After running the query and taking a close look at the dictionary, one will realize that:
</p> 1) Venues (first key of Response) is by itself another dictionary with 6 keys;
</p> 2) Out of these 6 keys, 5 are displayed as the 5 first columns of the dataframe;
</p> 3) The remaining key (location) is splitted into 12 columns because it is also a dictionary but with 12 keys.

#### Define information of interest and filter dataframe


In [11]:
col=0
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
print(col)
print(filtered_columns)
type(filtered_columns)

0
['name', 'categories', 'location.address', 'location.lat', 'location.lng', 'location.labeledLatLngs', 'location.distance', 'location.postalCode', 'location.cc', 'location.city', 'location.state', 'location.country', 'location.formattedAddress', 'location.crossStreet', 'id']


list

In [12]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns] #Access a group of rows and columns by label(s) or a boolean array.

# function that extracts the category of the venue because 'categories' is a list with a dictionary whose sole info of interest is name
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns] #split at '.' & gets last-right term [-1]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway,3fd66200f964a520f4e41ee3


In [13]:
dataframe_filtered.shape

(3, 15)

All this code eliminated two columns, renamed the remaining ones and extracted the info 'name' from a dict-inside-list associated with 'categories' Column. Categories is important as it is a keyword for searches ("Italian", etc).

#### Let's visualize the Italian restaurants that are nearby


In [14]:
dataframe_filtered.name

0          Harry's Italian Pizza Bar
1    Conca Cucina Italian Restaurant
2                               Ecco
Name: name, dtype: object

In [15]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel. It is an object

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
# zip creates a list of tuples as output. in this case (lat, lng, label(categories))
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<a id="item2"></a>


## 2. Explore a Given Venue

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**


### A. Let's explore the closest Italian restaurant -- _Harry's Italian Pizza Bar_


In [16]:
venue_id1 = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar that can be seen on "results" output
url1 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id1, CLIENT_ID, CLIENT_SECRET, VERSION)
url1

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=113ZWQSUYLUGMKP51IRZVZRVDHJBACXCYTXIXGD5GM5PVO5B&client_secret=5QXPGLDYT1SVGN3FVVLMZEV0EKJMYPMJNMJAMF2X3ZZEW0SP&v=20180604'

#### Send GET request for result


In [17]:
result1 = requests.get(url1).json()
list(result1.keys())
print()
print(result1['response']['venue'].keys())
print()
result1


dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])



{'meta': {'code': 200, 'requestId': '5fbfacfb11398a33e7350059'},
 'response': {'venue': {'id': '4fa862b3e4b0ebff2f749f06',
   'name': "Harry's Italian Pizza Bar",
   'contact': {'phone': '2126081007', 'formattedPhone': '(212) 608-1007'},
   'location': {'address': '225 Murray St',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.71521779064671,
      'lng': -74.01473940209351},
     {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
    'postalCode': '10282',
    'cc': 'US',
    'city': 'New York',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['225 Murray St',
     'New York, NY 10282',
     'United States']},
   'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
   'categories': [{'id': '4bf58dd8d48988d1ca941735',
     'name': 'Pizza Place',
     'pluralName': 'Pizza Places',
     'shortName': 'Pizza',
     'icon': {'prefix': 'https

### B. Let´s get the first restaurant´s overall rating, examine it and if deemed unsatisfactory, we wiil then choose the venue with the highest rating:

</p> "Try-Except" block works as a 'one-shot' verifier. It is more pratical than an "if-then-else" when it comes to look for a particular data or condition on a dictionaty as it provides a straight-to-the-point type of acess, rather than an iterative selection by decisions.


In [18]:
try:
    print(result1['response']['venue']['rating']) # this is a sequence of dict keys, from 'highest' to 'lower than'
except:
    print('This venue has not been rated yet.')

6.9


In [19]:
#Alex's Extra Effort for the sake of data explorarion only
result1['response']['venue']['tips']['count']

56

That is not a very good rating. Let's check the rating of the second closest Italian restaurant.


In [20]:
venue_id2 = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url2 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id2, CLIENT_ID, CLIENT_SECRET, VERSION)

result2 = requests.get(url2).json()
try:
    print(result2['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [21]:
#Alex's Extra Effort for the sake of data explorarion only
result2['response']['venue']['tips']['count']

0

Since this restaurant has no ratings, let's check the third restaurant.


In [22]:
venue_id3 = '3fd66200f964a520f4e41ee3' # ID of Ecco
url3 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id3, CLIENT_ID, CLIENT_SECRET, VERSION)

result3 = requests.get(url3).json()
try:
    print(result3['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.3


In [23]:
#Alex's Extra Effort for the sake of data explorarion only
result3['response']['venue']['tips']['count']

19

**Conclusion:** Since this restaurant has a slightly better rating, let's explore it further.

### C. Get the number of tips for the 3rd restaurant (Ecco):

In [24]:
print("The Restaurante we will look in detail is", result3['response']['venue']['name'],".")

The Restaurante we will look in detail is Ecco .


In [25]:
result3['response']['venue']['tips']['count']

19

### D. Get the Venue's Tips

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**

#### Create URL and send GET request. Make sure to set limit to get all tips


In [26]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips (it should be 19 at maximum)
url_3a = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id3, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results_tips = requests.get(url_3a).json()
results_tips

{'meta': {'code': 200, 'requestId': '5fbfae42f55f360d58242232'},
 'response': {'tips': {'count': 19,
   'items': [{'id': '5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 5,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'isSanctioned': False,
      'firstName': 'Nick',
      'lastName': 'E',
      'countryCode': 'US'},
     'authorInteractionType': 'liked'}]}}}

#### Get Tips and List of Tips´ Associated Features:

In [27]:
tips = results_tips['response']['tips']['items']   # similar to 'venues' at the 1st part of the session. It dekivers a list-type object
print(tips)
print("Type for 'tips' is:", type(tips))
print()

tip = results_tips['response']['tips']['items'][0] # 'tips' is a list from which we want to extract the first element (dict).
print(tip)
print("Type for 'tip' is:", type(tip))
print()

tip.keys()

[{'id': '5ab1cb46c9a517174651d3fe', 'createdAt': 1521601350, 'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites', 'type': 'user', 'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe', 'lang': 'en', 'likes': {'count': 0, 'groups': []}, 'logView': True, 'agreeCount': 5, 'disagreeCount': 0, 'todo': {'count': 0}, 'user': {'isSanctioned': False, 'firstName': 'Nick', 'lastName': 'E', 'countryCode': 'US'}, 'authorInteractionType': 'liked'}]
Type for 'tips' is: <class 'list'>

{'id': '5ab1cb46c9a517174651d3fe', 'createdAt': 1521601350, 'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites', 'type': 'user', 'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe', 'lang': 'en', 'likes': {'count': 

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

**Note:** the line 'tips = results_tips['response']['tips']['items']' works in a similar way of a Python composed line on which several commands are executed one after another. For this case, the object 'tips' will encompass every information that is contained within the dict key 'items' which is the "deepest" of all listed keys. In other words, it would not be possible to extract the quantity(count) of tips the restaurante was given because this data is outside 'items'. 

#### Format Column Width and display all Tips:

In [28]:
pd.set_option('display.max_colwidth', None)  # it was -1

tips_df = pd.json_normalize(tips) # json normalize tips to which I added 'pd.' due to the 'pink-message' suggestion

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName'] # there was user.id (????)
tips_filtered = tips_df.loc[:, filtered_columns] # here a 2nd df is built from 1st df data by selecting some columns and all rows [:]

# display tips
tips_filtered.reindex()

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",5,0,5ab1cb46c9a517174651d3fe,Nick,E


**Important:**  Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all 19 tips. That´s why the dict-output from url_3a search is so short.

<a id="item3"></a>


## 3. Search a Foursquare User in order to acess his-her Tips
</p> Another user - Not the User from previous Section (2D)

> `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**


### Define URL, send GET request and display features associated with user


In [29]:
idnumber = '48454224' # user ID with most agree counts and complete profile

url_user = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}'.format(idnumber, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL
print(url_user)
print()

# send GET request
results_user = requests.get(url_user).json()
print(results_user)
print()

#user_data = results['response']['user']
user_data = results_user['response']['tips']['items'][0]['venue']['photos']['groups'][0]['items']#['items']

# display features associated with user
#user_data.keys()
#results
pd.set_option('display.max_colwidth', None) # also with a -1 (original)

users_df = pd.json_normalize(user_data)

# filter columns
filtered_columns = ['id','user.firstName','user.lastName','suffix']  # it worked after elimination of 'user.id' from the second position
tips_filtered = users_df.loc[:, filtered_columns]

# display user's tips
df=tips_filtered
df
#url

https://api.foursquare.com/v2/users/48454224/tips?client_id=113ZWQSUYLUGMKP51IRZVZRVDHJBACXCYTXIXGD5GM5PVO5B&client_secret=5QXPGLDYT1SVGN3FVVLMZEV0EKJMYPMJNMJAMF2X3ZZEW0SP&v=20180604

{'meta': {'code': 200, 'requestId': '5fbfb11501f51442ca3456ba'}, 'response': {'tips': {'count': 1, 'items': [{'id': '518f4181498ed1b8ddf4d733', 'createdAt': 1368342913, 'text': 'El lugar esta muy bueno, pero el DJ nada más no!!!!!!', 'type': 'user', 'canonicalUrl': 'https://foursquare.com/item/518f4181498ed1b8ddf4d733', 'likes': {'count': 1, 'groups': [{'type': 'others', 'count': 1, 'items': []}], 'summary': '1 like'}, 'logView': True, 'agreeCount': 0, 'disagreeCount': 0, 'todo': {'count': 0}, 'venue': {'id': '50779500e4b0112fc267c1da', 'name': 'Tres Bar', 'location': {'lat': 20.17693758560229, 'lng': -98.04832423694748, 'labeledLatLngs': [{'label': 'display', 'lat': 20.17693758560229, 'lng': -98.04832423694748}], 'cc': 'MX', 'country': 'México', 'formattedAddress': ['México']}, 'categories': [{'id': '4bf

,id,user.firstName,user.lastName,suffix
0,57099084498e6e5b94d5a312,Cesar,H,/133773133_ODR5Au05ENkSyu3xxfV3VOfCa0idDfm9Q4n8YbhZDRQ.jpg
1,541e1d3b498e8e793290f7cd,Cesar,H,/30238440_mv7stX3aKbr8nHMI_dKkTCCUs3bhHwvARLuabvDIm5Y.jpg
2,53cb1c82498ef31f2f2eba34,Marcs,M,/51713900_PG5tGSnPloIvz--PJpC6cxDHmvfp6bA4wviH7d_rY_8.jpg


In [30]:
# I had to modify this because there is no information about 'id' inside the inner key 'user'. Therefore no 'user.id' is available.
g=df.loc[df['id'] == '57099084498e6e5b94d5a312']
print('First Name: ' + g['user.firstName'])
print('Last Name: ' + g['user.lastName'])

0    First Name: Cesar
Name: user.firstName, dtype: object
0    Last Name: H
Name: user.lastName, dtype: object


### Retrieve the User's Profile Image


In [31]:
# 1. grab prefix of photo
# 2. grab suffix of photo
# 3. concatenate them using the image size  
Image(url='https://fastly.4sqi.net/img/general/540x920/133773133_ODR5Au05ENkSyu3xxfV3VOfCa0idDfm9Q4n8YbhZDRQ.jpg')

Wow! So it turns out that Nick is a very active Foursquare user, with more than 250 tips.


### Get User's Tips

It seems this is the first user's (Nick E) tips for several venues. Actually, this notebook looks very confusing when it comes to a reasoning line and a logical sequence of activities. 

In [32]:
# define tips URL
user_id='484542633'
url_newuser = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results_newuser = requests.get(url_newuser).json()

# print(results_newuser) used it with visualization purposes...as the dict is too lenghty, I'd rather keep it not as active code

tips_newuser = results_newuser['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', None)
tips_df = pd.json_normalize(tips_newuser)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,They serve coffee!!!!!!,1,0,5accc98c0313204c9d7ec157
1,"Quick, cheap lunch that tastes good! Way shorter line than Chipotle, too.",2,0,5acbec70a0215b732e264fe8
2,You’re not a real New Yorker until you’ve shame-ordered Insomnia Cookies for delivery at 3am,1,0,5acbbd4eb1538e45373b07f5
3,Good for you yet still tasty! Clean green protein is my go-to after I hit the gym 💪,2,0,5acbbcda01235808d5d6dc75
4,Burger game strong 💪,1,0,5ab575fb6bdee65f759da8c1
5,"Great burgers & fries! Also, this place is exactly what it’s like when you go to a bar in the Southwest. Source: I’m from Arizona.",2,0,5ab5575d73fe2516ad8f363b
6,Açaí bowl + peanut butter + whey protein = 💪💪💪,1,0,5ab42db53c858d64af2688a4
7,Highly underrated and way less crowded than Central Park!,3,0,5ab42c396f706a29f53ad1a8
8,Get the açaí bowl with peanut butter after your work out and thank me later 👌,1,0,5ab42aca2a7ab6333652b266
9,"When you want a burger, this should be the first thing that comes to mind. A+!",1,0,5ab42a28da5e5617d18e3a6a


#### Let's get the Venue for the Tip with the Greatest Number of Agree Counts, that is, some sort of "Unanimity":

In [33]:
tip_id_great = '5ab5575d73fe2516ad8f363b' # tip id

# define URL
url_great_tip = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}'.format(idnumber, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET Request and examine results
result_great_tip = requests.get(url_great_tip).json()
result_great_tip

{'meta': {'code': 200, 'requestId': '5fbfb183d268be16693ed990'},
 'response': {'tips': {'count': 1,
   'items': [{'id': '518f4181498ed1b8ddf4d733',
     'createdAt': 1368342913,
     'text': 'El lugar esta muy bueno, pero el DJ nada más no!!!!!!',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/518f4181498ed1b8ddf4d733',
     'likes': {'count': 1,
      'groups': [{'type': 'others', 'count': 1, 'items': []}],
      'summary': '1 like'},
     'logView': True,
     'agreeCount': 0,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'venue': {'id': '50779500e4b0112fc267c1da',
      'name': 'Tres Bar',
      'location': {'lat': 20.17693758560229,
       'lng': -98.04832423694748,
       'labeledLatLngs': [{'label': 'display',
         'lat': 20.17693758560229,
         'lng': -98.04832423694748}],
       'cc': 'MX',
       'country': 'México',
       'formattedAddress': ['México']},
      'categories': [{'id': '4bf58dd8d48988d116941735',
        'name': 'Bar',


In [34]:
print(result_great_tip['response']['tips']['items'][0]['venue']['name'])
print()
print(result_great_tip['response']['tips']['items'][0]['venue']['location'])
print()
print("User Identification is:", idnumber)

Tres Bar

{'lat': 20.17693758560229, 'lng': -98.04832423694748, 'labeledLatLngs': [{'label': 'display', 'lat': 20.17693758560229, 'lng': -98.04832423694748}], 'cc': 'MX', 'country': 'México', 'formattedAddress': ['México']}

User Identification is: 48454224


**Note**: Take a look at the dict and look for (result_great_tip['response']['tips']['items'][0]['venue']['name'])...it is exactly Tres Bar. Same thing for second line which ends with ['location'] as it brings the whole dictionary associated with it. 

## 4. Explore a location

> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**


#### So, you just finished your gourmet dish at Ecco, and are just curious about the popular spots around the restaurant. In order to explore the area, let's start by getting the latitude and longitude values of Ecco Restaurant.


In [36]:
latitude_ecco = 40.715337
longitude_ecco = -74.008848

#### Define URL


In [37]:
url_pop = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_ecco, longitude_ecco, VERSION, radius, LIMIT)
url_pop

'https://api.foursquare.com/v2/venues/explore?client_id=113ZWQSUYLUGMKP51IRZVZRVDHJBACXCYTXIXGD5GM5PVO5B&client_secret=5QXPGLDYT1SVGN3FVVLMZEV0EKJMYPMJNMJAMF2X3ZZEW0SP&ll=40.715337,-74.008848&v=20180604&radius=500&limit=30'

#### Send GET request and examine results


In [38]:
import requests

In [39]:
results_pop = requests.get(url_pop).json()
results_pop

{'meta': {'code': 200, 'requestId': '5fbfb35c432e02660cf4c262'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Tribeca',
  'headerFullLocation': 'Tribeca, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 127,
  'suggestedBounds': {'ne': {'lat': 40.7198370045, 'lng': -74.00292208843348},
   'sw': {'lat': 40.710836995499996, 'lng': -74.01477391156652}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5d5f24ec09484500079aee00',
       'name': 'Los Tacos No. 1',
       'location': {'address': '136 Church St',
        'lat': 40.714267,
        'lng': -74.008756,
        'labeledLatLngs': [{'label': 'display',
         

In [40]:
'There are {} popular places to go around Ecco restaurant.'.format(len(results_pop['response']['groups'][0]['items']))

'There are 30 popular places to go around Ecco restaurant.'

**Note:** By now, it should be obvious that any developer must know the structure of any FourSquare output in details in order to work with these data. This last info (number of points of interest around Ecco) is easier to get, the deeper the knowledge of details the developer masters.

#### Get relevant part of JSON


In [49]:
items_pop = results_pop['response']['groups'][0]['items']
print(type(items_pop))
print()
items_pop[0]

<class 'list'>



{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5d5f24ec09484500079aee00',
  'name': 'Los Tacos No. 1',
  'location': {'address': '136 Church St',
   'lat': 40.714267,
   'lng': -74.008756,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.714267,
     'lng': -74.008756}],
   'distance': 119,
   'postalCode': '10007',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['136 Church St',
    'New York, NY 10007',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d151941735',
    'name': 'Taco Place',
    'pluralName': 'Taco Places',
    'shortName': 'Tacos',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/taco_',
     'suffix': '.png'},
    'primary': True}],
  'delivery': {'id': '2180700',
   'url': 'https://www.seamless.com/menu/los-tacos-no-1-tribeca-136-church-st-new-york/21

#### Process JSON and convert it into a Clean Dataframe (with only relevant Data):

In [51]:
dataframe_pop = pd.json_normalize(items_pop) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe_pop.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe_pop.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Los Tacos No. 1,Taco Place,136 Church St,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng': -74.008756}]",119,10007,US,New York,NY,United States,"[136 Church St, New York, NY 10007, United States]",NaN,NaN,5d5f24ec09484500079aee00
1,Korin,Furniture / Home Store,57 Warren St,40.714824,-74.009404,"[{'label': 'display', 'lat': 40.71482437714839, 'lng': -74.00940425461492}, {'label': 'entrance', 'lat': 40.714727, 'lng': -74.009399}]",73,10007,US,New York,NY,United States,"[57 Warren St (Church St), New York, NY 10007, United States]",Church St,Tribeca,4af5d65ff964a52091fd21e3
2,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,40.714788,-74.011132,"[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",202,10007,US,New York,NY,United States,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]",btwn Greenwich St & W Broadway,NaN,54148bc6498ea7bb8c05b70a
3,Takahachi Bakery,Bakery,25 Murray St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.713652845301894, 'lng': -74.0088038953017}, {'label': 'entrance', 'lat': 40.713716, 'lng': -74.008443}]",187,10007,US,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 10007, United States]",at Church St,NaN,4c154c9a77cea593c401d260
4,Takahachi,Sushi Restaurant,145 Duane St,40.716526,-74.008101,"[{'label': 'display', 'lat': 40.71652647412374, 'lng': -74.00810108466207}, {'label': 'entrance', 'lat': 40.716508, 'lng': -74.007989}]",146,10013,US,New York,NY,United States,"[145 Duane St (btwn W Broadway & Church St), New York, NY 10013, United States]",btwn W Broadway & Church St,NaN,4a8f2f39f964a520471420e3


In [54]:
dataframe_filtered.shape

(30, 16)

#### Let's visualize these items on the Map around our Location (Ecco Restaurant):

In [52]:
venues_map_pop = folium.Map(location=[latitude_ecco, longitude_ecco], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude_ecco, longitude_ecco],
    radius=10,
    popup='Ecco Restaurant',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map_pop)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map_pop)

# display map
venues_map_pop

<a id="item5"></a>


## 5. Explore Trending Venues

> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**


#### Now, instead of simply exploring the area around Ecco, you are interested in knowing the venues that are trending at the time you are done with your lunch, meaning the places with the highest foot traffic. So let's do that and get the trending venues around Ecco.


In [55]:
# define URL
url_trend = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_ecco, longitude_ecco, VERSION)

# send GET request and get trending venues
results_trend = requests.get(url_trend).json()
results_trend

{'meta': {'code': 200, 'requestId': '5fbfb7ca66ba7a282ba34e97'},
 'response': {'venues': []}}

### Check if any venues are trending at this time


In [62]:
if len(results_trend['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!. Year 2020 - The Year of Pandemics....'
    
else:
    trending_venues = results_trend['response']['venues']
    trending_venues_df = pd.json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [63]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!. Year 2020 - The Year of Pandemics....'

In [66]:
trending_venues_df.head

AttributeError: 'str' object has no attribute 'head'

**NOTES:** \
1) Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. \
2) I left the last error-message (pink block) to show that 'trending_venues_df' is a variable to which two different types can be cast:  'string' if there isn't any trending spots or 'pandas dataframe' if there is at least one trending place. A 'string'-type does  not show any 'head'.

### Visualize trending venues


In [64]:
if len(results_trend['response']['venues']) == 0:
    venues_map_trend = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map_trend = folium.Map(location=[latitude_ecco, longitude_ecco], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude_ecco, longitude_ecco],
        radius=10,
        popup='Ecco Restaurant',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map_trend)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map_trend)

In [65]:
# display map
venues_map_trend

'Cannot generate visual as no trending venues are available at the moment!'

<a id="item6"></a>


### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!

This notebook modified by Nayef Abou Tayoun ([https://www.linkedin.com/in/nayefaboutayoun/](https://www.linkedin.com/in/nayefaboutayoun?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ))


This notebook is part of a course on **Coursera** called _Applied Data Science Capstone_. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week2_LAB1).


<hr>
Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).
